# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.60,-73.97,77.00,53,2,4.74,US,1596290455
1,bluff,-46.60,168.33,52.34,81,100,11.48,NZ,1596290456
2,mount gambier,-37.83,140.77,51.80,87,90,8.05,AU,1596290456
3,rikitea,-23.12,-134.97,73.22,81,100,5.91,PF,1596290456
4,antofagasta,-23.65,-70.40,57.20,67,20,5.82,CL,1596290412


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

In [3]:
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating = False, max_intensity =300, point_radius =5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
narrowed_cities_df = cities_df[(cities_df["Max Temp"] <80) & (cities_df["Max Temp"] >70) & (cities_df["Wind Speed"] <10) \
                              & (cities_df["Cloudiness"]==0)].dropna(how = "any")
narrowed_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,karratha,-20.74,116.85,70.75,34,0,3.49,AU,1596290205
50,karoi,-16.81,29.69,73.26,20,0,9.15,ZW,1596290468
107,lagoa,39.05,-27.98,70.36,69,0,8.57,PT,1596290481
211,solwezi,-12.17,26.39,79.39,23,0,7.92,ZM,1596290419
264,howard springs,-12.50,131.05,71.60,78,0,3.36,AU,1596290518
267,richards bay,-28.78,32.04,78.53,41,0,8.30,ZA,1596290518
294,klyuchi,52.27,79.17,71.42,56,0,6.93,RU,1596290525
321,saldanha,-33.01,17.94,77.00,29,0,3.36,ZA,1596290531
333,binga,-17.62,27.34,75.92,23,0,8.32,ZW,1596290534
358,chiredzi,-21.05,31.67,77.81,29,0,7.94,ZW,1596290539


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = narrowed_cities_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
37,karratha,AU,-20.74,116.85,
50,karoi,ZW,-16.81,29.69,
107,lagoa,PT,39.05,-27.98,
211,solwezi,ZM,-12.17,26.39,
264,howard springs,AU,-12.50,131.05,
267,richards bay,ZA,-28.78,32.04,
294,klyuchi,RU,52.27,79.17,
321,saldanha,ZA,-33.01,17.94,
333,binga,ZW,-17.62,27.34,
358,chiredzi,ZW,-21.05,31.67,


In [6]:
params ={
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng= row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params= params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print("Missing field...skipping.")
hotel_df

,City,Country,Lat,Lng,Hotel Name
37,karratha,AU,-20.74,116.85,ibis Styles Karratha
50,karoi,ZW,-16.81,29.69,Bliss Lodge Karoi
107,lagoa,PT,39.05,-27.98,Casa Das Faias
211,solwezi,ZM,-12.17,26.39,Country Lodge Solwezi
264,howard springs,AU,-12.50,131.05,buffalo beach hostel
267,richards bay,ZA,-28.78,32.04,BON Hotel Waterfront Richards Bay
294,klyuchi,RU,52.27,79.17,Vesolaya Mayovka
321,saldanha,ZA,-33.01,17.94,Blue Bay Lodge
333,binga,ZW,-17.62,27.34,Masumu River Lodge
358,chiredzi,ZW,-21.05,31.67,The Nesbitt Arms


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))